In [14]:
using RigidBodyDynamics
using LinearAlgebra
using StaticArrays
using MeshCat
using MeshCatMechanisms
using MechanismGeometries

println("Libraries imported.")

Libraries imported.


In [16]:
# ------------------------------------------------------------------------
#                            MODEL DEFINITION
# ------------------------------------------------------------------------
src_dir = dirname(pathof(RigidBodyDynamics))
urdf_file = joinpath(src_dir, "..", "..", "..", "..", "..", "onr-dynamics-julia", "alphaArmFixedJaws.urdf")
mechanism = parse_urdf(urdf_file)

visuals = URDFVisuals(urdf_file)

println("URDF parsed. \n")

URDF parsed. 



In [17]:
# ------------------------------------------------------------------------
#                              CONTROLLER
# ------------------------------------------------------------------------

base_joint, shoulder_joint, elbow_joint, wrist_joint = joints(mechanism)

function simple_control!(torques::AbstractVector, t, state::MechanismState)
    torques[velocity_range(state, base_joint)] .= 2.0*sin(t)
    torques[velocity_range(state, shoulder_joint)] .= 0.
    torques[velocity_range(state, elbow_joint)] .= 0.
    torques[velocity_range(state, elbow_joint)] .= 0.
end;

In [18]:
# ------------------------------------------------------------------------
#                              SIMULATION
# ------------------------------------------------------------------------
state = MechanismState(mechanism)
zero_velocity!(state)
set_configuration!(state, base_joint, 0.0)
set_configuration!(state, shoulder_joint, 0.0)
set_configuration!(state, elbow_joint, 0.0)
set_configuration!(state, wrist_joint, 0.0)

final_time = 5.
ts, qs, vs = simulate(state, final_time, simple_control!; Δt = 1e-3)

println("Simulation finished.")

Simulation finished.


In [21]:
# ------------------------------------------------------------------------
#                             VISUALIZATION
# ------------------------------------------------------------------------
# vis = Visualizer()
# open(vis)

# delete!(vis)
mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf_file))
# set_configuration!(mvis, [0.0, 0.0])
open(mvis)
MeshCatMechanisms.animate(mvis, ts, qs; realtimerate = 1.)

# animation = Animation(mvis, ts, qs)
# setanimation!(mvis, animation)

println("\n done.")

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://localhost:8702
└ @ MeshCat /home/hkolano/.julia/packages/MeshCat/nXUuG/src/visualizer.jl:69


MethodError: [91mMethodError: no method matching range(::Int64, ::Int64; step=0.25)[39m
[91m[0mClosest candidates are:[39m
[91m[0m  range(::Any; length, stop, step) at range.jl:76[39m